# Titanic - Preprocessing Data


## Preparation

### Import necessary libraries

In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


### Load train.csv file

In [2]:

data_path = os.path.join("..", "..", "data")
titanic = pd.read_csv(os.path.join(data_path, "raw", "train.csv"))
df = titanic.copy()
test_dataset = titanic = pd.read_csv(os.path.join(data_path, "raw", "test.csv"))
df_test = titanic.copy()

print("Successfully load training data.")
df.head()

Successfully load training data.


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
for x in ['Pclass', 'Sex', 'Embarked']:
    df[x] = df[x].astype('category')
    df_test[x] = df_test[x].astype('category')
df['Survived'] = df['Survived'].astype('category')

In [4]:
print("===== Number of missing values (train dataset) =====\n")
print(df[['Age', 'Cabin', 'Embarked']].isnull().sum())
print('\n')
print("===== Number of missing values (test dataset) =====\n")
print(df_test[['Age', 'Cabin', 'Fare']].isnull().sum())

===== Number of missing values (train dataset) =====

Age         177
Cabin       687
Embarked      2
dtype: int64


===== Number of missing values (test dataset) =====

Age       86
Cabin    327
Fare       1
dtype: int64


In [5]:
print("===== Percentage of missing values on train dataset =====\n")
print(df[['Age', 'Cabin', 'Embarked']].isnull().sum() / len(df) * 100)
print('\n')
print("===== Percentage of missing values on test dataset =====\n")
print(df_test[['Age', 'Cabin', 'Fare']].isnull().sum() / len(df_test) * 100)

===== Percentage of missing values on train dataset =====

Age         19.865320
Cabin       77.104377
Embarked     0.224467
dtype: float64


===== Percentage of missing values on test dataset =====

Age      20.574163
Cabin    78.229665
Fare      0.239234
dtype: float64


## Handle missing value

We need to find solutions to handle missing value.
From the output detecting missing value, here's the proportion of missing value for `Age`, `Cabin`, and `Embarked` in train dataset:

- Age: 19.87%
- Cabin: 77.10%
- Embarked: 0.22%

For test dataset, here's the proportion of missing value for `Age`, `Cabin`, and `Fare`:
- Age: 20.57%
- Cabin: 78.23%
- Fare: 0.24%

For `Cabin` features, although the missing value percentage are pretty high, it could be a good information to show passenger's room level, which is a really crucial detail related to survivability. Furthermore, we can base on that information to know if the passengers stayed at the front part (bow) of the Titanic, which is the part of the ship that sank first.

For `Age` features, it could related to the survivability, as children and elderly usually be prioritized.

For `Embarked` features, it could related to passenger's room level (passengers go on the ship first might have room that are in higher level).

Because of that, I think that we should not remove these features. We need to handle value instead.

For testing dataset, we also have missing value in `Fare` feature. We will handle it.

### Handle missing age value

Honorific title inside passengers' name could tell us a little about their age.

At first, we splited value `Age` into 5 main groups: Mr, Mrs, Master, Miss, and other in order to not only impute missing values but also capture potential age-related patterns.

But after looking at survivability rate of each group, we decided not to complicate things and just impute missing value with median age of the whole dataset. Except for `Master` group, we will impute missing value with median age of this group as title "Master" is only for people under 12.

In [6]:
master = df['Name'].str.contains(r',\s*Master.', regex=True)
df_master = df[master].copy()

mean_age = df['Age'].median()
mean_age_master = df_master['Age'].median()

df_master['Age'] = df_master['Age'].fillna(mean_age_master)
df[master] = df_master

df['Age'] = df['Age'].fillna(mean_age)
df_test['Age'] = df_test['Age'].fillna(mean_age)

### Handle missing values from Cabin, Embarked (train dataset) and Fare (test dataset)

Perhaps the only features that we could use to guess people's cabin is Ticket number, but since there are no ticket number system, we could not conclude anything from that.

At first, I thought it would be possible to fill missing `Cabin` value by looking at passengers' ticket class, fare, and embarked point, but after some research, I found out that there is no clear relationship between these features and `Cabin` value.

Therefore, I will just remove the `Cabin` feature, as it has too many missing values and we could not find a way to fill them.

As for Embarked value, I will choose "S" to fill that, as the primary embarked point was Southampton (Titanic trip started from there)

As for Fare value in testing dataset, I will fill it with median of Fare value.

In [7]:
df = df.drop(['Cabin'], axis=1)
df_test = df_test.drop(['Cabin'], axis=1)

df['Embarked'] = df['Embarked'].fillna("S")
df_test['Embarked'] = df_test['Embarked'].fillna("S")

fare_mean = df['Fare'].median()
df_test['Fare'] = df_test['Fare'].fillna(fare_mean)

In [8]:

print("===== Quantity of missing values =====")
print(df[['Age', 'Embarked']].isnull().sum())
print(df_test[['Fare']].isnull().sum())
print("\n===== Percentage of missing values =====")
print(df[['Age', 'Embarked']].isnull().sum() / len(df) * 100)
print(df_test[['Fare']].isnull().sum() / len(df_test) * 100)

===== Quantity of missing values =====
Age         0
Embarked    0
dtype: int64
Fare    0
dtype: int64

===== Percentage of missing values =====
Age         0.0
Embarked    0.0
dtype: float64
Fare    0.0
dtype: float64


## Detecting outliers

This section will mainly focus on addressing outlier values in numeric features.

Based on the outputs from the boxplot from EDA Analysis, we could see that `Age`, `Fare`, `SibSp`, and `Parch` features have outlier values. However, since `SibSp` and `Parch` features are not continuous (both features have a limited number of unique values), we will not handle outlier values in these two features, as it could affect negatively on the model.

To handle outlier values, IQR method is used to find the lower bound and upper bound, then replace outlier values with values within these bounds using techniques such as capping or imputation.

In [9]:
num_cols = ['Age', 'Fare']
def detect_outliers_iqr(df, col, factor=1.5):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = max(Q1 - factor * IQR, 0)  # Age cannot be negative
    upper_bound = Q3 + factor * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    # print(f'{col} - Outliers (IQR): {len(outliers)}, Lower: {lower_bound:.2f}, Upper: {upper_bound:.2f}')
    return outliers, lower_bound, upper_bound


for col in num_cols:
    outliers, lower_bound, upper_bound = detect_outliers_iqr(df, col)
    print(f'Total outliers in {col}: {len(outliers)}')

Total outliers in Age: 66
Total outliers in Fare: 116


Base on the outcome, we could see that:
- Total outliers in Age: 66
- Total outliers in Fare: 116

The number of outliers in `Fare` is quite high, we will have to handle it carefully in the next part.

## Feature encoding

In [10]:
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    int64   
 1   Survived     891 non-null    category
 2   Pclass       891 non-null    category
 3   Name         891 non-null    object  
 4   Sex          891 non-null    category
 5   Age          891 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    object  
 9   Fare         891 non-null    float64 
 10  Embarked     891 non-null    category
dtypes: category(4), float64(2), int64(3), object(2)
memory usage: 52.8+ KB


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,S


In [11]:
df['Sex'] = df['Sex'].cat.codes
df['Embarked'] = df['Embarked'].cat.codes
df_test['Sex'] = df_test['Sex'].cat.codes
df_test['Embarked'] = df_test['Embarked'].cat.codes
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",1,22.0,1,0,A/5 21171,7.2500,2
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,38.0,1,0,PC 17599,71.2833,0
2,3,1,3,"Heikkinen, Miss. Laina",0,26.0,0,0,STON/O2. 3101282,7.9250,2
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,35.0,1,0,113803,53.1000,2
4,5,0,3,"Allen, Mr. William Henry",1,35.0,0,0,373450,8.0500,2


## Save file

We will save train and test files to use for the future

In [12]:
df.to_csv(os.path.join(data_path, "preprocessed", "preprocessed_train.csv"), index=False)
df_test.to_csv(os.path.join(data_path, "preprocessed", "preprocessed_test.csv"), index=False)
print("Successfully saved preprocessed data.")

Successfully saved preprocessed data.


# The end